In [1]:
import pandas as pd
from shopifyapi import ShopifyApp
import os
from dotenv import load_dotenv
import re
import csv
import numpy as np
from converter import *

In [2]:
load_dotenv()

True

# Support Functions

In [44]:
def adding_upc(df):
    try:
        upc_df = pd.read_csv('available_upc.csv')
        upc_df.fillna('', inplace=True)
        for index in range(len(df)):
            if pd.notna(df.loc[index, 'Variant SKU']):
                df.loc[index, 'Variant Barcode'] = upc_df[upc_df['Available'] == ''].iloc[0, 0].copy()
                df.loc[index, 'Google Shopping / MPN'] = upc_df[upc_df['Available'] == ''].iloc[0, 0].copy()
                index = upc_df.index[upc_df['Available'] == ''][0]
                upc_df.iloc[index, 1] = 'Used'
            else:
                df.loc[index, 'Variant Barcode'] = ''
                df.loc[index, 'Google Shopping / MPN'] = ''
        upc_df.to_csv('available_upc.csv', index=False)

        return df
    
    except IndexError:
        print('UPC are not available!')

In [45]:
def adding_tags(title, tags):
    try:
        if pd.notna(title):
            if pd.notna(tags):
                result = tags + ', toys'
            else:
                result = 'toys'
        else:
            result = None
        
        return result
    except Exception as e:
        print(e)
            

In [46]:
def clean_text(text):
    if pd.isna(text):
        return text
    # First replace words containing 'yescomusa' with 'Trendtimes'
    intermediate_text = re.sub(r'\w*yescomusa\w*', 'Trendtimes', text, flags=re.IGNORECASE)
    
    # Then remove 'dti direct' (case insensitive)
    cleaned_text = re.sub(r'thelashop', 'Trendtimes', intermediate_text, flags=re.IGNORECASE)
    
    return cleaned_text

In [47]:
def clean_handle(text):
    if pd.isna(text):
        return text
        
    text = re.sub(r'yescomusa', 'trendtimes', text, flags=re.IGNORECASE)
    text = re.sub(r'yescom', 'trendtimes', text, flags=re.IGNORECASE)
    
    text = re.sub(r'thelashop', 'trendtimes', text, flags=re.IGNORECASE)
    
    return text

In [48]:
def clean_text_2(text):
    if pd.isna(text):
        return text
    
    text = str(text)
    
    # Define replacement rules (case-insensitive)
    replacements = {
        r'\byescomusa\b': 'Trendtimes',
        r'\bthelashop\b': 'Trendtimes'
    }
    
    # Apply all replacements in one pass
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

    # Define replacement rules (case-insensitive)
    replacements2 = {
        r'\w*yescom\w*': 'Trendtimes',
        r'\w*lashop\w*': 'Trendtimes',
    }
    
    # Apply all replacements in one pass
    for pattern, replacement in replacements2.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    
    text = text.strip()
    
    return text if text else pd.NA

In [49]:
def remove_video_embeds(text):
    if pd.isna(text):
        return text
    
    # pattern = r"<h2>[^<]*?video[^<]*?</h2>\s*<p>.*?<iframe.*?</iframe>\s*</p>"
    pattern = r"<h2>[^<]*?video[^<]*?</h2>\s*(<p>\s*<iframe.*?</iframe>\s*</p>|<iframe.*?</iframe>)"
    cleaned_text = re.sub(pattern, "", text, flags=re.DOTALL | re.IGNORECASE)
    
    return cleaned_text


In [50]:
def remove_manual_embeds(text):
    if pd.isna(text):
        return text
    
    # pattern = r"<h2>[^<]*?video[^<]*?</h2>\s*<p>.*?<iframe.*?</iframe>\s*</p>"
    # manual_pattern = r"<h2[^>]*>Documents</h2>\s*<table[^>]*>.*?</table>|<table[^>]*>(?:[^<]*|<(?!table)[^>]*>)*?(?:Manual|manual)(?:[^<]*|<(?!table)[^>]*>)*?</table>"
    # cleaned_text = re.sub(manual_pattern, "", text, flags=re.DOTALL | re.IGNORECASE)

     # Create a regex pattern to match content between the start and end markers
    pattern = re.escape("<h2>Documents</h2>") + r".*?" + re.escape("</table>")
    # Remove the block using regex
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
    
    return cleaned_text

In [51]:
def reduce_price(price):
    if pd.isna(price):
        return price
    float_price = float(price)
    if (price is None) or (float_price == 0) or (price == '0.00'):
        result = "0.00"
    else:
        result = float_price - round(float_price * 2 / 100, 2)

    return f"{result:.2f}"

In [52]:
def modify_sku(sku, warranty, plate):
    if pd.isna(sku):
        return sku
    if warranty == 'None - $0':
        if plate == 'None - $0':
            return sku
        elif plate == 'Custom license plate - $39':
            return sku + '-P'
    elif warranty == '1 year - $89':
        if plate == 'None - $0':
            return sku + '-W'
        elif plate == 'Custom license plate - $39':
            return sku + '-WP'

In [53]:
def modify_price(price, warranty, plate):
    if pd.isna(price):
        return price
    price = float(price)
    if warranty == 'None - $0':
        if plate == 'None - $0':
            return price
        elif plate == 'Custom license plate - $39':
            return price + 39.00
    elif warranty == '1 year - $89':
        if plate == 'None - $0':
            return price + 89.00
        elif plate == 'Custom license plate - $39':
            return price + 39.00 + 89.00

In [54]:
def shift_option(df):
    for index in range(len(df)):
        if pd.isna(df.loc[index, 'Option1 Value']):
            df.loc[index, 'Option1 Name'] = df.loc[index, 'Option2 Name']
            df.loc[index, 'Option1 Value'] = df.loc[index, 'Option2 Value']
            df.loc[index, 'Option2 Name'] = df.loc[index, 'Option3 Name']
            df.loc[index, 'Option2 Value'] = df.loc[index, 'Option3 Value']
            df.loc[index, 'Option3 Name'] = ''
            df.loc[index, 'Option3 Value'] = ''
        else:
            pass
    
    return df    

# Read Data

## Yescomusa

In [104]:
df_ycm = pd.read_csv('data/yescomusa_products.csv')

## Thelashop

In [105]:
df_las = pd.read_csv('data/thelashop_products.csv')

# Remove yescomusa and thelashop word

## Yescomusa

In [106]:
df_ycm[df_ycm['Handle'].str.contains('yescom', case=False, na=False)] #.to_csv('data/cek.csv', index=False)

,Handle,Title,Body (HTML),Vendor,Product Category,Type,Tags,Published,Option1 Name,Option1 Value,...,Variant Weight Unit,Variant Tax Code,Cost per item,Included / United States,Price / United States,Compare At Price / United States,Included / International,Price / International,Compare At Price / International,Status
2076,yescom-artificial-grass-turf-synthetic-carpet-...,Yescom Artificial Grass Turf Synthetic Carpet ...,<div class=&quot;tab-content collapse-content ...,Yescom,Patio & Garden > Outdoor Decor > Artificial Turf,Artificial Turf,"Patio & Garden, Outdoor Decor, Artificial Turf",True,NaN,NaN,...,lb,NaN,199.79,True,NaN,NaN,NaN,NaN,NaN,active
2077,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2078,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2079,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2080,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2081,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2082,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2083,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2084,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2085,yescom-artificial-grass-turf-synthetic-carpet-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df_ycm['Body (HTML)'] = df_ycm['Body (HTML)'].apply(clean_text_2)

In [108]:
df_ycm['Title'] = df_ycm['Title'].apply(clean_text_2)

In [109]:
df_ycm['Handle'] = df_ycm['Handle'].apply(clean_handle)

## TheLaShop

In [110]:
df_las[df_las['Handle'].str.contains('lashop', case=False, na=False)] #.to_csv('data/cek.csv', index=False)

,Handle,Title,Body (HTML),Vendor,Product Category,Type,Tags,Published,Option1 Name,Option1 Value,...,Variant Weight Unit,Variant Tax Code,Cost per item,Included / United States,Price / United States,Compare At Price / United States,Included / International,Price / International,Compare At Price / International,Status
1276,thelashop-4-machines-tattoo-kit-w-power-supply...,TheLAShop 4 Tattoo Guns Machines Tattoo Kit w/...,<div class=&quot;rte&quot;><meta charset=&quot...,TheLAShop,Health & Beauty,Tattoo Kit,Health & Beauty,True,NaN,NaN,...,lb,NaN,34.79,True,NaN,NaN,NaN,NaN,NaN,active
1277,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1278,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1280,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1281,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1282,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1283,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1284,thelashop-4-machines-tattoo-kit-w-power-supply...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df_las['Body (HTML)'] = df_las['Body (HTML)'].apply(clean_text_2)

In [112]:
df_las['Title'] = df_las['Title'].apply(clean_text_2)

In [113]:
df_las['Handle'] = df_las['Handle'].apply(clean_handle)

# Merge Product

In [95]:
las_checker = df_las[~df_las['Title'].isin([np.nan, ''])][['Title']]

In [96]:
las_checker['is_duplicated'] = 1

In [97]:
las_checker.nunique()

Title            1547
is_duplicated       1
dtype: int64

In [98]:
df_ycm = df_ycm.merge(las_checker, how='left', on='Title')

In [99]:
duplicated_handles = df_ycm[df_ycm['is_duplicated'] == 1]['Handle'].unique().tolist()

In [100]:
len(duplicated_handles)

0

In [101]:
duplicated_handles

[]

In [64]:
df_ycm.loc[df_ycm['Handle'].isin(duplicated_handles), 'is_duplicated'] = 1

In [66]:
cleaned_df_ycm = df_ycm[df_ycm['is_duplicated'] != 1]

In [67]:
cleaned_df_ycm.drop(columns='is_duplicated', inplace=True)

# Get Collection

In [114]:
df = pd.concat([df_ycm, df_las], ignore_index=True)

In [71]:
df['Collection Rule'] = df['Tags'].str.split(',')

In [73]:
parent_product_df = df[~pd.isna(df['Title'])]

In [74]:
exploded_parent_product_df = parent_product_df.explode('Collection Rule', ignore_index=True)

In [75]:
exploded_parent_product_df.tail()

,Handle,Title,Body (HTML),Vendor,Product Category,Type,Tags,Published,Option1 Name,Option1 Value,...,Price / United States,Compare At Price / United States,Included / International,Price / International,Compare At Price / International,Status,is_duplicated_x,is_duplicated_y,is_duplicated,Collection Rule
6687,5ft-led-spiral-christmas-tree-usb-powered,TheLAShop 5ft LED Spiral Christmas Tree USB Po...,<div class=&quot;rte&quot;><p>TheLAShop 5ft LE...,TheLAShop,Lamps & Lighting > Outside Lights,Outdoor String Lights,"Lamps & Lighting, Outside Lights",True,Color,RGB,...,NaN,NaN,NaN,NaN,NaN,active,NaN,NaN,1.0,Outside Lights
6688,25ft-telescoping-flagpole-kit-with-eagle-ball-...,TheLAShop Sectional Flagpole with Light Solar ...,<div class=&quot;rte&quot;><p>Boost your patri...,TheLAShop,Home & Garden > Outdoor Living,Flag Poles,"Home & Garden, Outdoor Living",True,Size,20ft,...,NaN,NaN,NaN,NaN,NaN,active,NaN,NaN,1.0,Home & Garden
6689,25ft-telescoping-flagpole-kit-with-eagle-ball-...,TheLAShop Sectional Flagpole with Light Solar ...,<div class=&quot;rte&quot;><p>Boost your patri...,TheLAShop,Home & Garden > Outdoor Living,Flag Poles,"Home & Garden, Outdoor Living",True,Size,20ft,...,NaN,NaN,NaN,NaN,NaN,active,NaN,NaN,1.0,Outdoor Living
6690,5-ft-clear-led-lighted-spiral-christmas-tree-m...,TheLAShop 5ft LED Lighted Spiral Christmas Tre...,<div class=&quot;rte&quot;><p>This <strong>5ft...,TheLAShop,Lamps & Lighting > Outside Lights,Outdoor String Lights,"Lamps & Lighting, Outside Lights",True,Color,Cool White,...,NaN,NaN,NaN,NaN,NaN,active,NaN,NaN,1.0,Lamps & Lighting
6691,5-ft-clear-led-lighted-spiral-christmas-tree-m...,TheLAShop 5ft LED Lighted Spiral Christmas Tre...,<div class=&quot;rte&quot;><p>This <strong>5ft...,TheLAShop,Lamps & Lighting > Outside Lights,Outdoor String Lights,"Lamps & Lighting, Outside Lights",True,Color,Cool White,...,NaN,NaN,NaN,NaN,NaN,active,NaN,NaN,1.0,Outside Lights


In [76]:
exploded_parent_product_df['Collection Rule'] = exploded_parent_product_df['Collection Rule'].str.strip()

In [77]:
exploded_parent_product_df['Collection Name'] = exploded_parent_product_df['Collection Rule'].str.title()

In [78]:
collection_df = exploded_parent_product_df[['Collection Name', 'Collection Rule']]

In [79]:
collection_df.dropna(inplace=True, ignore_index=True)

/tmp/ipykernel_14127/3537422268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collection_df.dropna(inplace=True, ignore_index=True)


In [80]:
collection_df.drop_duplicates(inplace=True, ignore_index=True)

/tmp/ipykernel_14127/707458884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collection_df.drop_duplicates(inplace=True, ignore_index=True)


In [81]:
collection_df

,Collection Name,Collection Rule
0,Patio & Garden,Patio & Garden
1,Patio Umbrella & Shade,Patio Umbrella & Shade
2,Canopy Gazebo Replacements,Canopy Gazebo Replacements
3,Camping & Outdoor,Camping & Outdoor
4,Camp Shelters,Camp Shelters
...,...,...
226,Outdoor Living,Outdoor Living
227,Kitchen & Bath,Kitchen & Bath
228,Auto Parts & Accessories,Auto Parts & Accessories
229,Furniture & Decor,Furniture & Decor


In [82]:
collection_df.to_csv('data/yscomusa and thelashop collection.csv', index=False) 

# Shopify

In [85]:
shopify = ShopifyApp(store_name=os.getenv('STORE_NAME'), access_token=os.getenv('ACCESS_TOKEN'))
client = shopify.create_session()

Creating session...


In [86]:
for index in range(len(collection_df)):
    print(collection_df.iloc[index, 0])

Patio & Garden
Patio Umbrella & Shade
Canopy Gazebo Replacements
Camping & Outdoor
Camp Shelters
Pop Up Canopies
Fitness & Sports
Trampoline & Accessories
Treadmills
Musical Instrument
Record Players & Turntables
Outdoor Decor
Flag Poles
Patio Umbrella
Market Umbrella
Car Awnings
Lit Umbrellas & Lights
Shade Sail
Beauty & Health
Tattoo Kits
Awning Canopy
Pole Dancing Pole
Pool Supplies
Health Care
Saunas
Umbrella Replacement Canopy
Home
Kitchen & Dining
Commercial Electric Griddle
Detox Foot Bath
Home Improvement
Kitchen Faucet & Sink
Business & Industrial
Trade Show Display
Grommet Machine
Deep Fryers
Jack Hammers
Photo & Live Stream
Speakers
Portable Camping Toilet
Massager
Makeup & Nail Art
Cosmetology Rolling Case
Keyboard Instruments
Food Warmers
String Instruments
Household Appliances
Air Cooler
Automotive
Carbon Fiber Vinyl
Vehicle Cargo Racks
Archery Bows
Home Decor
Dj & Partying
Smart Lighting
Medical
Mobility & Disability
Mobility Aids
Pergola Replacement Canopy
Mayo Stand
Cy

In [87]:
has_next_page = True
cursor = None
results = list()
while has_next_page:
    response = shopify.get_collections(client, cursor=cursor)
    records = response['data']['collections']['nodes']
    results.extend(records)
    has_next_page = response['data']['collections']['pageInfo']['hasNextPage']
    cursor = response['data']['collections']['pageInfo']['endCursor']
results_df = pd.DataFrame.from_records(results)
results_df.to_csv('data/collection_list.csv', index=False)

Getting collection list...
<Response [200 OK]>
{'data': {'collections': {'nodes': [{'handle': 'barbie', 'id': 'gid://shopify/Collection/293575360569', 'title': 'Barbie'}, {'handle': 'beauty-the-beast', 'id': 'gid://shopify/Collection/293575393337', 'title': 'Beauty & The Beast'}, {'handle': 'brady-bunch', 'id': 'gid://shopify/Collection/293575426105', 'title': 'Brady Bunch'}, {'handle': 'cinderella', 'id': 'gid://shopify/Collection/293575458873', 'title': 'Cinderella'}, {'handle': 'disney', 'id': 'gid://shopify/Collection/293575491641', 'title': 'Disney'}, {'handle': 'ironman', 'id': 'gid://shopify/Collection/293575524409', 'title': 'Ironman'}, {'handle': 'shrek', 'id': 'gid://shopify/Collection/293575557177', 'title': 'Shrek'}, {'handle': 'sleeping-beauty', 'id': 'gid://shopify/Collection/293575589945', 'title': 'Sleeping Beauty'}, {'handle': 'snow-white', 'id': 'gid://shopify/Collection/293575622713', 'title': 'Snow White'}, {'handle': 'tangled-rapunzel', 'id': 'gid://shopify/Collect

In [89]:
shopify_col_df = pd.read_csv('data/collection_list.csv')

In [92]:
collection_df['Handle'] = collection_df.apply(lambda x: to_handle(title=x['Collection Name'], alt_title=''), axis=1)

/tmp/ipykernel_14127/4215949206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collection_df['Handle'] = collection_df.apply(lambda x: to_handle(title=x['Collection Name'], alt_title=''), axis=1)


In [93]:
collection_df = collection_df.merge(shopify_col_df, how='left', left_on='Handle', right_on='handle')

In [94]:
collection_df

,Collection Name,Collection Rule,Handle,handle,id,title
0,Patio & Garden,Patio & Garden,patio-garden,NaN,NaN,NaN
1,Patio Umbrella & Shade,Patio Umbrella & Shade,patio-umbrella-shade,NaN,NaN,NaN
2,Canopy Gazebo Replacements,Canopy Gazebo Replacements,canopy-gazebo-replacements,NaN,NaN,NaN
3,Camping & Outdoor,Camping & Outdoor,camping-outdoor,NaN,NaN,NaN
4,Camp Shelters,Camp Shelters,camp-shelters,NaN,NaN,NaN
...,...,...,...,...,...,...
226,Outdoor Living,Outdoor Living,outdoor-living,NaN,NaN,NaN
227,Kitchen & Bath,Kitchen & Bath,kitchen-bath,NaN,NaN,NaN
228,Auto Parts & Accessories,Auto Parts & Accessories,auto-parts-accessories,auto-parts-accessories,gid://shopify/Collection/293661737017,Auto Parts & Accessories
229,Furniture & Decor,Furniture & Decor,furniture-decor,furniture-decor,gid://shopify/Collection/293663309881,Furniture & Decor


In [95]:
collection_df[~pd.isna(collection_df['id'])]

,Collection Name,Collection Rule,Handle,handle,id,title
9,Musical Instrument,Musical Instrument,musical-instrument,musical-instrument,gid://shopify/Collection/293646008377,Musical Instrument
12,Flag Poles,Flag Poles,flag-poles,flag-poles,gid://shopify/Collection/293663178809,Flag Poles
17,Shade Sail,Shade Sail,shade-sail,shade-sail,gid://shopify/Collection/293666750521,Shade Sail
34,Grommet Machine,Grommet Machine,grommet-machine,grommet-machine,gid://shopify/Collection/293663572025,Grommet Machine
43,Keyboard Instruments,Keyboard Instruments,keyboard-instruments,keyboard-instruments,gid://shopify/Collection/293663932473,Keyboard Instruments
55,Medical,Medical,medical,medical,gid://shopify/Collection/293588860985,Medical
59,Mayo Stand,Mayo Stand,mayo-stand,mayo-stand,gid://shopify/Collection/293629001785,Mayo Stand
71,Salon Supplies & Equipment,Salon Supplies & Equipment,salon-supplies-equipment,salon-supplies-equipment,gid://shopify/Collection/293666553913,Salon Supplies & Equipment
80,Manicures & Pedicures,Manicures & Pedicures,manicures-pedicures,manicures-pedicures,gid://shopify/Collection/293664292921,Manicures & Pedicures
83,Pet Playpen,Pet Playpen,pet-playpen,pet-playpen,gid://shopify/Collection/293664915513,Pet Playpen


In [97]:
collection_df[pd.isna(collection_df['id'])].to_csv('data/not available collection.csv', index=False)

## Data import

In [115]:
products_df = pd.concat([df_las, df_ycm], ignore_index=True)

In [117]:
products_df[pd.notna(products_df['Variant SKU'])][['Handle','Title','Variant SKU', 'Vendor']]

,Handle,Title,Variant SKU,Vendor
0,spinning-static-dancing-pole-dance-removable-e...,Trendtimes Spinning Static Dance Pole Portable...,LAS000323,TheLAShop
10,2-guns-series-tattoo-kit-machine-stinger,Trendtimes 2 Tattoo Machines Tattoo Kit w/ Pow...,LAS000549,TheLAShop
19,tattoo-kit-8-guns-12-wrap-coil-lcd-power-suppl...,Trendtimes Tattoo Kit 8 Machines LCD Power Sup...,LAS003153,TheLAShop
29,large-bronze-rolling-hair-stylist-salon-makeup...,Trendtimes Rolling Makeup Station Hair Nail Wo...,LAS000997,TheLAShop
45,large-bronze-rolling-hair-stylist-salon-makeup...,NaN,LAS003528,NaN
...,...,...,...,...
30946,trade-show-display-folding-board-header-gray,Trendtimes Trade Show Display Folding Board He...,YES1804,Yescom
30950,shampoo-bowl-drain-hose-1-7-16-dia,"Trendtimes Shampoo Bowl Drain Hose 1 7/16"" Dia.",YES1065,Yescom
30956,8-5x11-menu-covers-set4-for-restaurant-8-view-...,Trendtimes 8.5x11 Menu Covers Set(4) for Resta...,YES3409,Yescom
30966,outdoor-umbrella-stand-steel-parasol-base-34lb...,Trendtimes Outdoor Umbrella Stand Steel Paraso...,YES3408,Yescom


## Data Report

In [98]:
products_data = products_df[pd.notna(products_df['Variant SKU'])]

NameError: name 'products_df' is not defined

In [ ]:
products_data.loc[products_data.duplicated('Variant SKU', keep='first'), 'is_duplicated'] = 1

In [ ]:
products_data[products_data['is_duplicated'] == 1][['Handle','Variant SKU', 'Vendor']]

In [ ]:
products_data.to_csv('products_data.csv', index=False)

# Adding Tags

In [ ]:
products_df = pd.read_csv('product_data/products_with_upc.csv')

In [ ]:
products_df['Tags'] = products_df.apply(lambda x: adding_tags(x['Title'], x['Tags']), axis=1)

In [ ]:
products_df[['Handle','Title','Tags', 'Vendor']]

# Change Vendor

In [ ]:
products_df['Vendor']

In [ ]:
products_df['Vendor'] = products_df['Vendor'].apply(lambda x: 'Htrn71' if x == 'Hobbytron' else None)

In [ ]:
products_df['Vendor']

In [ ]:
products_df.to_csv('products_with_upc_upd_vendor_tags_rev1.csv', index=False)

# Fill null option value

In [ ]:
product_df = pd.read_csv('product_data/products_with_upc_upd_vendor_tags_rev1.csv')

In [ ]:
product_df

In [ ]:
product_df.loc[(pd.notna(product_df['Option1 Name'])) & (pd.isna(product_df['Option1 Value'])), ['Option1 Value']] = 'No Battery'

In [ ]:
product_df[(pd.notna(product_df['Option3 Name'])) & (pd.isna(product_df['Option3 Value']))]

In [ ]:
product_df.info()

In [ ]:
product_df.loc[pd.notna(product_df['Title']), 'Google Shopping / Custom Label 0'] = 'Htrn71'

In [ ]:
product_df.loc[pd.notna(product_df['Title']), 'Google Shopping / Custom Label 0']

In [ ]:
product_df.to_csv('products_with_upc_upd_vendor_tags_rev2.csv', index=False)

# Remove Embeded Video from body html

In [ ]:
products_df['Body (HTML)'] = products_df['Body (HTML)'].apply(remove_video_embeds)

In [ ]:
products_df['Body (HTML)'] = products_df['Body (HTML)'].apply(remove_manual_embeds)

In [ ]:
products_df.iloc[34,2]

In [ ]:
products_df[products_df['Handle'].str.contains('magic-cars', case=False, na=False)]

In [ ]:
products_df.iloc[112,1]

# Reduce Price

In [ ]:
products_df['Variant Price'] = products_df['Variant Price'].apply(reduce_price)

# Generate Inventory csv

In [ ]:
inventory_df = products_df[['Handle', 'Title', 'Option1 Name', 'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value', 'Variant SKU', 'Variant Inventory Qty']].copy()

In [ ]:
inventory_df = inventory_df[pd.notna(inventory_df['Variant SKU'])]

In [ ]:
inventory_df['L']

# Add Warranty and Custom Plate Option

In [ ]:
products_df.loc[pd.notna(products_df['Title']), 'Option2 Name'] = 'Warranty' 

In [ ]:
products_df['Option2 Value'] = products_df.apply(lambda x: ['None - $0', '1 year - $89'] if pd.notna(x['Variant SKU']) else '', axis=1)

In [ ]:
products_df.loc[pd.notna(products_df['Title']), 'Option3 Name'] = 'Custom license plate' 

In [ ]:
products_df['Option3 Value'] = products_df.apply(lambda x: ['None - $0', 'Custom license plate - $39'] if pd.notna(x['Variant SKU']) else '', axis=1)

In [ ]:
products_df = products_df.explode('Option2 Value')

In [ ]:
with open('variant_unused_columns.csv', 'r') as file:
			rows = csv.reader(file)
			variant_unused_columns = [row[0] for row in rows]

In [ ]:
products_df.loc[products_df.duplicated(['Handle', 'Option1 Value'], keep='first'), variant_unused_columns] = ''

In [ ]:
products_df = products_df.explode('Option3 Value')

In [ ]:
products_df.loc[products_df.duplicated(['Handle', 'Option2 Value'], keep='first'), variant_unused_columns] = ''

In [ ]:
products_df.loc[products_df['Handle']=='12v-magic-cars-kids-cruiser-1-seater-motorcycle', ['Handle','Title', 'Image Src', 'Option1 Name', 'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value']]

# Modify Variant SKU

In [ ]:
products_df['Variant SKU_1'] = products_df.apply(lambda x: modify_sku(x['Variant SKU'], x['Option2 Value'], x['Option3 Value']), axis=1)

In [ ]:
products_df.drop(columns='Variant SKU', inplace=True)

In [ ]:
products_df.rename({'Variant SKU_1': 'Variant SKU'}, axis=1, inplace=True)

# Modify Variant Price

In [ ]:
products_df['Variant Price_1'] = products_df.apply(lambda x: modify_price(x['Variant Price'], x['Option2 Value'], x['Option3 Value']), axis=1)

In [ ]:
products_df.drop(columns='Variant Price', inplace=True)

In [ ]:
products_df.rename({'Variant Price_1': 'Variant Price'}, axis=1, inplace=True)

# Adding UPC

In [118]:
products_df.reset_index(drop=True, inplace=True)

In [119]:
products_df[pd.notna(products_df['Variant SKU'])][['Handle', 'Variant SKU']]

,Handle,Variant SKU
0,spinning-static-dancing-pole-dance-removable-e...,LAS000323
10,2-guns-series-tattoo-kit-machine-stinger,LAS000549
19,tattoo-kit-8-guns-12-wrap-coil-lcd-power-suppl...,LAS003153
29,large-bronze-rolling-hair-stylist-salon-makeup...,LAS000997
45,large-bronze-rolling-hair-stylist-salon-makeup...,LAS003528
...,...,...
30946,trade-show-display-folding-board-header-gray,YES1804
30950,shampoo-bowl-drain-hose-1-7-16-dia,YES1065
30956,8-5x11-menu-covers-set4-for-restaurant-8-view-...,YES3409
30966,outdoor-umbrella-stand-steel-parasol-base-34lb...,YES3408


In [120]:
products_df = adding_upc(products_df)

/tmp/ipykernel_15444/4203351286.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'Variant Barcode'] = ''
/tmp/ipykernel_15444/4203351286.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'Google Shopping / MPN'] = ''


In [ ]:
# Checking
products_df[products_df['Handle']=='12v-magic-cars-kids-cruiser-1-seater-motorcycle'][['Handle','Title','Variant SKU', 'Variant Price', 'Variant Barcode', 'Image Src','Option1 Name', 'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value']].to_csv('cek.csv', index=False)

In [ ]:
products_df = shift_option(products_df)

In [121]:
products_df.to_csv('data/final_products.csv', index=False)

In [110]:
title_test = 'TheLAShop BV250 4/4 Full Size Maple Wood Advanced Violin & Case'
cleaned_title = clean_text_2(title_test)
print(cleaned_title)

Trendtimes BV250 4/4 Full Size Maple Wood Advanced Violin & Case


# Adding Collection Id

In [4]:
df_col = pd.read_csv('data/not available collection.csv')
df_ref = pd.read_csv('data/existing_collection_list.csv')

In [11]:
df_col_with_id = pd.merge(left=df_col, right=df_ref, how='left', left_on='Collection Name', right_on='title')

In [12]:
df_col_with_id

,Collection Name,Collection Rule,Handle,handle,id,title
0,Patio & Garden,Patio & Garden,patio-garden,patio-garden,gid://shopify/Collection/306593955897,Patio & Garden
1,Patio Umbrella & Shade,Patio Umbrella & Shade,patio-umbrella-shade,patio-umbrella-shade,gid://shopify/Collection/306593988665,Patio Umbrella & Shade
2,Canopy Gazebo Replacements,Canopy Gazebo Replacements,canopy-gazebo-replacements,canopy-gazebo-replacements,gid://shopify/Collection/306594021433,Canopy Gazebo Replacements
3,Camping & Outdoor,Camping & Outdoor,camping-outdoor,camping-outdoor,gid://shopify/Collection/306594054201,Camping & Outdoor
4,Camp Shelters,Camp Shelters,camp-shelters,camp-shelters,gid://shopify/Collection/306594086969,Camp Shelters
...,...,...,...,...,...,...
197,Tools & Hardware,Tools & Hardware,tools-hardware,tools-hardware,gid://shopify/Collection/306600411193,Tools & Hardware
198,Interior Lights,Interior Lights,interior-lights,interior-lights,gid://shopify/Collection/306600443961,Interior Lights
199,Outdoor Living,Outdoor Living,outdoor-living,outdoor-living-1,gid://shopify/Collection/306600476729,Outdoor Living
200,Kitchen & Bath,Kitchen & Bath,kitchen-bath,kitchen-bath,gid://shopify/Collection/306600509497,Kitchen & Bath


In [13]:
df_col_with_id.to_csv('data/collection_publication_data.csv', index=False)